对大模型调用错误的记录进行查询，尝试进行重试

In [58]:
import pandas as pd


def load_excel(file_path: str) -> pd.DataFrame:
    """
    读取Excel文件，验证必要列并处理空值
    要求文件必须包含"商品ID"和"商品名称"列（可修改required_cols适配实际列名）
    """
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        required_cols = ['商品ID', '商品名称']
        # 检查必要列是否存在
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"缺少必要列：{', '.join(missing_cols)}，需包含{required_cols}")
        # 去除商品ID或名称为空的无效行
        df = df.dropna(subset=required_cols).reset_index(drop=True)
        # 强制转为字符串类型，避免数字ID/名称拼接出错（如科学计数法、格式丢失）
        df['商品ID'] = df['商品ID'].astype(str).str.strip()
        df['商品名称'] = df['商品名称'].astype(str).str.strip()
        return df
    except Exception as e:
        print(f"读取文件{file_path}失败：{str(e)}")
        raise

In [66]:
import math
import pandas as pd
import re


def pandas_str_to_series(s) -> pd.Series:
    # s为None或""或nan
    if s is None or s == "" or (isinstance(s, float) and math.isnan(s)):
        return None

    inner = s[s.find("(") + 1: s.rfind(")")]

    pattern = re.compile(r"(\w+)=('[^']*'|[^,]*)")
    data = {k: v.strip("'") if v.strip() != "nan" else None for k, v in pattern.findall(inner)}

    # 3️⃣ 转为 DataFrame
    df = pd.DataFrame([data])
    return df.iloc[0]



In [72]:

# 加载数据表文件
df = load_excel("../../output/补充Top3相似商品.xlsx")
df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品
0,22639130381,【规格可选】30W充电套装 适用苹果8-14系列PD快充充电头 苹果手机数据线充电器 1个,单独 1米线,6978410500134,6.75,6.75,NaN,3,所搜商品,所搜商品,...,NaN,NaN,NaN,NaN,直插充电器,41430057383,"Pandas(商品ID='15408186991', 商品名称='iPhone16_15苹果...","Pandas(商品ID='15394518849', 商品名称='苹果iPhone13_14...",NaN,NaN
1,22638543544,农夫山泉 饮用纯净水 550ml12瓶_包,550ml*12,6921168560424,9.90,22.00,NaN,34,推荐,推荐,...,NaN,NaN,NaN,NaN,包装饮用水,41429308607,"Pandas(商品ID='18766628564', 商品名称='【农夫山泉】饮用天然水 5...","Pandas(商品ID='18766628564', 商品名称='【农夫山泉】饮用天然水 5...","Pandas(商品ID='20440643609', 商品名称='【农夫山泉】 饮用天然水 ...","Pandas(商品ID='18766628564', 商品名称='【农夫山泉】饮用天然水 5..."
2,22636514960,红牛Red Bull 维生素风味饮料 250ml_罐,250ml*1罐,6970640429988,8.07,8.23,NaN,7,推荐,推荐,...,NaN,NaN,NaN,NaN,运动功能饮料（非健字号）,41430928019,"Pandas(商品ID='19220893481', 商品名称='红牛Red Bull 维生...",NaN,NaN,"Pandas(商品ID='19220893481', 商品名称='红牛Red Bull 维生..."
3,22640202093,百岁山 饮用天然矿泉水 570ml_瓶,NaN,06922255466476,2.99,2.99,NaN,6,推荐,推荐,...,NaN,NaN,NaN,NaN,天然矿泉水,41431324722,"Pandas(商品ID='10908200588', 商品名称='百岁山 饮用天然水矿泉水 ...",NaN,NaN,"Pandas(商品ID='10908200588', 商品名称='百岁山 饮用天然水矿泉水 ..."
4,22740184269,康师傅茉莉蜜茶调味茶饮品 1000ml_瓶,1000ml*1瓶,6920459991985,5.54,5.65,NaN,1,推荐,推荐,...,NaN,NaN,NaN,NaN,茶饮料,41669396232,"Pandas(商品ID='10906519970', 商品名称='康师傅 茉莉清茶调味茶饮品...","Pandas(商品ID='21147266243', 商品名称='康师傅 青梅绿茶果味茶饮品...","Pandas(商品ID='10908201285', 商品名称='康师傅 茉莉蜜茶花茶调味花...","Pandas(商品ID='10908201285', 商品名称='康师傅 茉莉蜜茶花茶调味花..."


In [110]:
# 提取相似商品列值为error的记录，相似商品列可能为None
error_rows = df[df['相似商品'].notnull() & df['相似商品'].str.contains('error')]
error_rows

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品


In [100]:
# error_rows数据信息，多少行，多少列
error_rows.shape

(8, 29)

In [101]:
from Limter import RateLimiter

# 设置限流器
limiter = RateLimiter()

In [71]:
from llm_match import process_llm_row, llm_rank

# 迭代错误记录error_rows, 尝试进行重试
for index, row in error_rows.iterrows():
    print(f"正在处理错误记录：{index} - {row['商品名称']}")
    await limiter.record_call(800)
    # 重新解析相似商品1（ID-名称）列为Series对象，值为str,举例：Pandas(商品ID='10908200588', 商品名称='百岁山 饮用天然水矿泉水 570ml_瓶', 规格='570ml*1瓶', 条码=6922255451427.0, 折扣价=3.0, 原价=3.5, 活动=nan, 销售=72, 店内一级分类='水丨饮料', 店内二级分类='全部', 图片='http://p0.meituan.net/shangouproductapi/c...
    similar_product1 = pandas_str_to_series(row['相似商品1（ID-名称）'])
    similar_product2 = pandas_str_to_series(row['相似商品2（ID-名称）'])
    similar_product3 = pandas_str_to_series(row['相似商品3（ID-名称）'])
    # print(similar_product1['商品名称'])

    # 重新进行模型匹配判断
    # similar_product1为None，则为""
    top1 = similar_product1['商品名称'] if similar_product1 is not None else ""
    top2 = similar_product2['商品名称'] if similar_product2 is not None else ""
    top3 = similar_product3['商品名称'] if similar_product3 is not None else ""
    idx = await llm_rank(row.商品名称, top1, top2, top3)
    print(f"匹配结果：{idx}")
    # 更新error_rows相似商品字段
    if idx == 1:
        error_rows.at[index, '相似商品'] = error_rows.loc[index, '相似商品1（ID-名称）']
    elif idx == 2:
        error_rows.at[index, '相似商品'] = error_rows.loc[index, '相似商品2（ID-名称）']
    elif idx == 3:
        error_rows.at[index, '相似商品'] = error_rows.loc[index, '相似商品3（ID-名称）']
    elif idx == 0:
        error_rows.at[index, '相似商品'] = None
    else:
        error_rows.at[index, '相似商品'] = 'error'
        print("匹配结果错误")



正在处理错误记录：10 - 康师傅 冰力十足冰红茶柠檬口味茶饮品 500ml_瓶
正在处理： 康师傅 冰力十足冰红茶柠檬口味茶饮品 500ml_瓶
匹配结果：0
正在处理错误记录：15 - 香飘飘蜜谷 桃桃红柚果汁茶饮料 400ml_杯
正在处理： 香飘飘蜜谷 桃桃红柚果汁茶饮料 400ml_杯
匹配结果：0
正在处理错误记录：17 - 三得利 0能量0糖清香微甜茉莉乌龙茶饮料 500ml_瓶
正在处理： 三得利 0能量0糖清香微甜茉莉乌龙茶饮料 500ml_瓶
匹配结果：1
正在处理错误记录：23 - 王老吉 凉茶植物饮料 500ml_瓶
正在处理： 王老吉 凉茶植物饮料 500ml_瓶
匹配结果：1
正在处理错误记录：31 - 可口可乐 畅爽装汽水 680ml_瓶
正在处理： 可口可乐 畅爽装汽水 680ml_瓶
匹配结果：0
正在处理错误记录：35 - 乌苏 11°P乌苏啤酒罐装 500ml_听
正在处理： 乌苏 11°P乌苏啤酒罐装 500ml_听
匹配结果：0
正在处理错误记录：38 - 得宝 抽取式面巾纸 90抽_袋
正在处理： 得宝 抽取式面巾纸 90抽_袋
匹配结果：0
正在处理错误记录：40 - 百事可乐 可乐型汽水 330ml_罐
正在处理： 百事可乐 可乐型汽水 330ml_罐
匹配结果：1
正在处理错误记录：58 - 陶华碧老干妈 固形物不低于50%风味鸡油辣椒 280g_瓶
正在处理： 陶华碧老干妈 固形物不低于50%风味鸡油辣椒 280g_瓶
匹配结果：0
正在处理错误记录：60 - 李锦记薄盐生抽酿造酱油 500ml_瓶
正在处理： 李锦记薄盐生抽酿造酱油 500ml_瓶
商品：李锦记薄盐生抽酿造酱油 500ml_瓶 发生异常: 1 validation error for RankSelect
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "rank_index": 0\n  \t\t\t\n}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invali

CancelledError: 

In [102]:
error_rows.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品
876,22638784830,百事食品 乐事 得克萨斯烧烤味原切马铃薯片 135g_袋,135g*1袋,6924743924147,12.50,12.50,NaN,0,休闲零食,全部,...,NaN,NaN,NaN,NaN,薯片/薯条,41432253268,"Pandas(商品ID='21400214587', 商品名称=""百事食品乐事Lay's 青...","Pandas(商品ID='10908897246', 商品名称='乐事 得克萨斯烧烤味 40...","Pandas(商品ID='22482849173', 商品名称=""百事食品乐事Lay's 黄...",error
975,22636916951,小滑头 素食经典薄辣片调味面制品 18g_袋,18g*1袋,6928822300358,0.01,0.89,0.11折 限1份,6,肉干辣条,全部,...,NaN,NaN,NaN,NaN,面筋制品,41431660476,"Pandas(商品ID='22528543585', 商品名称='佳龙 甜辣味五谷杂粮调味面...","Pandas(商品ID='21605348637', 商品名称='霸王丝 辣丝青豆辣丝 18...",NaN,error
2620,22639486253,NL8420狂神兵乓球6只装,10克*1份,6933509184206,11.24,11.24,NaN,0,运动游泳,全部,...,NaN,NaN,NaN,NaN,球类,41431531063,"Pandas(商品ID='21148974841', 商品名称='杰士邦 零感超薄沁薄避孕套...",NaN,NaN,error
3688,22639245179,多场景可用！透明胶带1卷 4.5厘米60米快递打包封口胶带强粘性封箱胶带宽胶带,透明胶带1卷,NaN,4.97,4.97,NaN,0,行李收纳,全部,...,NaN,NaN,NaN,NaN,胶带,41431370651,"Pandas(商品ID='10907345890', 商品名称='45米大号 封箱胶带 透明...","Pandas(商品ID='10908591309', 商品名称='200米大号 封箱胶带 透...","Pandas(商品ID='10907345890', 商品名称='45米大号 封箱胶带 透明...",error
4153,22638376764,纳米双面胶一卷 透明防水无痕耐高温亚克力胶带水洗魔力胶带,厚2mm宽3CM长1M,NaN,4.18,4.18,NaN,0,办公学习,全部,...,NaN,NaN,NaN,NaN,胶带,41431453523,"Pandas(商品ID='10906639907', 商品名称='强力透明胶带 水洗无痕透明...","Pandas(商品ID='10908897204', 商品名称='3M双面胶汽车用强力耐高温...","Pandas(商品ID='17986344253', 商品名称='【贴对联专用胶带】双面胶透...",error


In [56]:
# 导出结果excel
error_rows.to_excel("../../output/补充Top3相似商品d1b5681b-df78-438e-9371-df272cb4f633_error.xlsx", index=False)

In [103]:
import asyncio
import pandas as pd
from llm_match import process_llm_row, llm_rank

async def handle_error_row(index, row, limiter, error_rows):
    print(f"正在处理错误记录：{index} - {row['商品名称']}")
    
    await limiter.record_call(800)  # 限流

    # 解析相似商品
    similar_product1 = pandas_str_to_series(row['相似商品1（ID-名称）'])
    similar_product2 = pandas_str_to_series(row['相似商品2（ID-名称）'])
    similar_product3 = pandas_str_to_series(row['相似商品3（ID-名称）'])

    # 获取商品名称
    top1 = similar_product1['商品名称'] if similar_product1 is not None else ""
    top2 = similar_product2['商品名称'] if similar_product2 is not None else ""
    top3 = similar_product3['商品名称'] if similar_product3 is not None else ""

    # 模型匹配
    idx = await llm_rank(row.商品名称, top1, top2, top3)
    print(f"匹配结果：{idx}")

    # 更新 DataFrame
    if idx == 1:
        error_rows.at[index, '相似商品'] = row['相似商品1（ID-名称）']
    elif idx == 2:
        error_rows.at[index, '相似商品'] = row['相似商品2（ID-名称）']
    elif idx == 3:
        error_rows.at[index, '相似商品'] = row['相似商品3（ID-名称）']
    elif idx == 0:
        error_rows.at[index, '相似商品'] = None
    else:
        error_rows.at[index, '相似商品'] = 'error'
        print("匹配结果错误")

async def process_error_rows(error_rows, limiter, batch_size=10):
    tasks = []
    for index, row in error_rows.iterrows():
        tasks.append(handle_error_row(index, row, limiter, error_rows))

        # 分批提交任务，避免一次性提交过多协程
        if len(tasks) >= batch_size:
            await asyncio.gather(*tasks)
            tasks = []

    # 处理剩余的任务
    if tasks:
        await asyncio.gather(*tasks)

# 调用方式
# asyncio.run(process_error_rows(error_rows, limiter))


In [107]:
await process_error_rows(error_rows, limiter)

正在处理错误记录：975 - 小滑头 素食经典薄辣片调味面制品 18g_袋
正在处理错误记录：5130 - 牛皮纸袋手提袋 多规格可选打包外卖餐饮袋服装袋送人礼品袋
正在处理： 小滑头 素食经典薄辣片调味面制品 18g_袋
正在处理： 牛皮纸袋手提袋 多规格可选打包外卖餐饮袋服装袋送人礼品袋
匹配结果：2
匹配结果：3


In [96]:
error_rows

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品
349,22636833600,北冰洋橙汁汽水 330毫升6听 塑包装,330毫升*6听,6972020770178,30.00,30.00,NaN,0,饮料专区,全部,...,NaN,NaN,NaN,NaN,碳酸饮料,41428346449,"Pandas(商品ID='10907427129', 商品名称='【6瓶装】雪碧 清爽柠檬味...","Pandas(商品ID='19951061439', 商品名称='麒麟 11°P一番搾啤酒 ...","Pandas(商品ID='21309388195', 商品名称='胖东来 12°P精酿小麦啤...",None
500,22636514957,维他奶 0胆固醇低脂无负担原味豆奶 250ml6盒_提,250ml*6盒,4891028700074,67.00,67.00,NaN,0,牛奶乳品,全部,...,NaN,NaN,NaN,NaN,植物/复合蛋白饮料,41430928016,"Pandas(商品ID='10908897630', 商品名称='维他奶 巧克力味豆奶饮料 ...","Pandas(商品ID='19163529377', 商品名称='维他奶 原味豆奶 早餐奶 ...","Pandas(商品ID='19163726972', 商品名称='【单盒】维他奶原味豆奶25...","Pandas(商品ID='19163529377', 商品名称='维他奶 原味豆奶 早餐奶 ..."
559,22640208359,粮小子福乐回头 蓝莓牛奶味面包 80g_袋,80g*1袋,6950140300253,2.80,2.80,NaN,1,面包糕点,蛋糕糕点,...,NaN,NaN,NaN,NaN,面包,41430470543,"Pandas(商品ID='12282018058', 商品名称='福乐回头 3+2紫米夹心面...","Pandas(商品ID='20731286023', 商品名称='迪味臻 泡椒凤爪山椒味 8...","Pandas(商品ID='10906992864', 商品名称='嘉顿 牛奶味威化饼干 20...",None
604,22637353101,百草味 固形物不低于60%水果捞黄桃罐头 312g_罐,312g*1罐,6948939666642,13.92,14.20,NaN,0,泡面速食,全部,...,NaN,NaN,NaN,NaN,水果罐头,41430022160,"Pandas(商品ID='18286159538', 商品名称='欢乐家 黄桃罐头 460g...","Pandas(商品ID='18286159538', 商品名称='欢乐家 黄桃罐头 460g...","Pandas(商品ID='18803074679', 商品名称='欢乐家 黄桃罐头 900g...",None
876,22638784830,百事食品 乐事 得克萨斯烧烤味原切马铃薯片 135g_袋,135g*1袋,6924743924147,12.50,12.50,NaN,0,休闲零食,全部,...,NaN,NaN,NaN,NaN,薯片/薯条,41432253268,"Pandas(商品ID='21400214587', 商品名称=""百事食品乐事Lay's 青...","Pandas(商品ID='10908897246', 商品名称='乐事 得克萨斯烧烤味 40...","Pandas(商品ID='22482849173', 商品名称=""百事食品乐事Lay's 黄...",error
975,22636916951,小滑头 素食经典薄辣片调味面制品 18g_袋,18g*1袋,6928822300358,0.01,0.89,0.11折 限1份,6,肉干辣条,全部,...,NaN,NaN,NaN,NaN,面筋制品,41431660476,"Pandas(商品ID='22528543585', 商品名称='佳龙 甜辣味五谷杂粮调味面...","Pandas(商品ID='21605348637', 商品名称='霸王丝 辣丝青豆辣丝 18...",NaN,error
1039,22639803069,劲仔 糖醋味深海小鱼 12g_袋,NaN,6951957205793,1.46,1.49,NaN,0,肉干辣条,全部,...,NaN,NaN,NaN,NaN,鱼肉类制品,41428632870,"Pandas(商品ID='11025282035', 商品名称='劲仔小鱼 深海小鱼糖醋味 ...","Pandas(商品ID='11023957514', 商品名称='劲仔小鱼 深海小鱼烧烤味 ...","Pandas(商品ID='11023957514', 商品名称='劲仔小鱼 深海小鱼烧烤味 ...","Pandas(商品ID='11025282035', 商品名称='劲仔小鱼 深海小鱼糖醋味 ..."
1054,22653446605,周长江 小豆条豆制熟食 16g_袋,16g*1袋,6927301200486,0.77,0.79,NaN,3,肉干辣条,全部,...,NaN,NaN,NaN,NaN,豆制品,41451588320,"Pandas(商品ID='11998560831', 商品名称='味芝元 香辣鱼尾 16g_...","Pandas(商品ID='20396363876', 商品名称='缺牙齿 香辣味素牛肚 16...","Pandas(商品ID='20396363876', 商品名称='缺牙齿 香辣味素牛肚 16...",None
1324,22639891009,陶华碧老干妈 固形物不低于50%风味鸡油辣椒 280g_瓶,280g*1瓶,6921804700795,11.90,15.89,NaN,1,粮油调味,全部,...,NaN,NaN,NaN,NaN,辣酱,41431262747,"Pandas(商品ID='10908360026', 商品名称='老干妈 风味鸡油辣椒 28...","Pandas(商品ID='10908360026', 商品名称='老干妈 风味鸡油辣椒 28...","Pandas(商品ID='10908744389', 商品名称='老干妈 风味豆豉油制辣椒 ...",None
1426,22639020443,维达Vinda 3层超韧系列纸面巾 130抽_袋,130抽*1袋,6901236341582,4.40,4.49,NaN,0,生活用纸,全部,...,NaN,NaN,NaN,NaN,抽纸,41430983027,"Pandas(商品ID='10908978843', 商品名称='维达抽纸3层130抽6包超...","Pandas(商品ID='12010882438', 商品名称='维达 超韧抽纸纸巾 抽取面...","Pandas(商品ID='17852133347', 商品名称='【山姆】MM 经典纸面巾 ...","Pandas(商品ID='10908978843', 商品名称='维达抽纸3层130抽6包超..."


In [108]:
# 将error_rows数据 按照index更新相似商品列到df中，使用at
for index, row in error_rows.iterrows():
    df.at[index, '相似商品'] = row['相似商品']

In [109]:
# 导出excel
df.to_excel("../../output/补充Top3相似商品retry4.xlsx", index=False)